<a href="https://colab.research.google.com/github/YashDeepp/Movie-Review-Classification-BERT/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 53.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 41.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), 25.78 MiB | 5.64 MiB/s, done.


In [ ]:
data_train=pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx")
data_test=pd.read_excel("/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx")

In [ ]:
(x_train,y_train),(x_test,y_test),preproc = text.texts_from_df(train_df=data_train,
                                                               text_column='Reviews',
                                                               label_columns='Sentiment',
                                                               val_df=data_test,
                                                               maxlen=500,
                                                               preprocess_mode='bert')

['neg', 'pos']
   neg  pos
0  1.0  0.0
1  1.0  0.0
2  1.0  0.0
3  1.0  0.0
4  1.0  0.0
['neg', 'pos']
   neg  pos
0  0.0  1.0
1  0.0  1.0
2  1.0  0.0
3  0.0  1.0
4  1.0  0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [ ]:
model=text.text_classifier(name='bert',
                          train_data=(x_train,y_train),
                          preproc=preproc)


Is Multi-Label? False
maxlen is 500


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [ ]:
learner=ktrain.get_learner(model=model,train_data=(x_train,y_train),
                   val_data=(x_test,y_test),
                   batch_size=6)
#If max length is 500 then batch size must be 6

In [ ]:
#Find out best learning rate
#learner.lr_find()
#learner.plot()
#it may take many days
learner.fit_onecycle(lr=2e-5,epochs=1)



begin training using onecycle policy with max lr of 2e-05...
2387/4167 [================>.............] - ETA: 23:34 - loss: 0.2932 - accuracy: 0.8740

KeyboardInterrupt: ignored

In [ ]:
predictor=ktrain.get_predictor(learner.model, preproc)

In [ ]:
data=['this movie was horrible, the plot was very bad',
      'this movie was great , good acting']

In [ ]:
predictor.predict(data,return_proba=True )

array([[0.9966934 , 0.00330662],
       [0.02652964, 0.97347033]], dtype=float32)

In [ ]:
#Saving
predictor.save('/content/bert')

In [ ]:
!zip -r /content/bert.zip /content/bert

  adding: content/bert/ (stored 0%)
  adding: content/bert/tf_model.h5 (deflated 11%)
  adding: content/bert/tf_model.preproc (deflated 48%)


In [ ]:
predictor_load=ktrain.load_predictor('/content/bert')

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [ ]:
predictor_load.get_classes()

['neg', 'pos']